# Network growth
This notebook is based on the notebooks of Michael Szell "03_poi_based_generation.ipynb" and "04_analyze_results.ipynb" that you can find here:

https://github.com/mszell/bikenwgrowth

### Parameters

In [ ]:
debug = True # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Network growth

In [ ]:
# params
delta = 300
alpha = 0.5
Ds = list(np.arange(5000,95000,5000)) # m added to the infrastructure
delta_and_alpha = [delta,alpha]
limited_abstract_edges = None # set limited_abstract_edges meters != None (e.g. 5000 meters) for big cities

In [ ]:
# Load networks
G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')
G_carallsimplified = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall_simplified')
G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')
G_biketracksimplified = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack_simplified')
    
# Load seed points
with open(PATH["data"] + placeid + "/" + placeid + '_seed-points_delta' + str(delta) + '.csv') as f:
    nnids = [int(line.rstrip()) for line in f]

# initialize weights
POIs_weights = {}
for i in range(len(nnids)):
    POIs_weights[str(nnids[i])] = 1.0

In [ ]:
# Generation
(GTs, GT_abstracts,BWs) = greedy_triangulation_routing(G_carall, nnids, Ds,POIs_weights, True, G_biketrack, delta_and_alpha, limited_abstract_edges)

## Analyze network

In [ ]:
# Analyze the existing bicycle infrastructure

output_ex, covs_ex, covs_ex_acc = calculate_metrics_additively([G_biketrack], [G_biketracksimplified], [0], G_carall, nnids, buffer_walk, buffer_accident, numnodepairs, True,output = {
            "directness_lcc": [],
            "coverage": [],
            "accidents_coverage19": [],
            "trips_coverage": [],
            "components": []
            },detour=0)

In [ ]:
# Analyze our proposed network

print(placeid + ": Analyzing results")

# output contains lists for all the Ds values of the corresponding metric
output, cov, covs_acc = calculate_metrics_additively(GTs, GT_abstracts, Ds, G_carall, nnids, buffer_walk, buffer_accident, numnodepairs, debug,output = {
            "directness_lcc": [],
            "coverage": [],
            "accidents_coverage19": [],
            "trips_coverage": [],
            "components": []
            },detour = detour, cov_prev = covs_ex[0], cov_prev_acc = covs_ex_acc[0])

In [ ]:
# Save network results
results = {"placeid": placeid, "Ds": Ds, "GTs": GTs, "GT_abstracts": GT_abstracts}
write_result(results, "pickle", placeid, 'delta' + str(delta) + "NetGrowth_alpha" + str(alpha) + ".pickle")

# Save metrics results to csv 
write_result(output, "dict", placeid,  'delta' + str(delta) + "NetGrowth_alpha" + str(alpha) +".csv")

# Save metrics results to csv (existing bicycle infrastructure) 
write_result(output_ex, "dict", placeid,  "results_existing_infra.csv")